In [1]:
# Upload the dataset zip file
from google.colab import files
uploaded = files.upload()  # Upload the zip file when prompted

# Find the filename
import zipfile
import os

for filename in uploaded.keys():
    print("Uploaded file:", filename)
    zip_path = filename

# Extract the uploaded zip file
extract_dir = "/content/text_emotion_data"
os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("Files extracted to:", extract_dir)

Saving archive.zip to archive.zip
Uploaded file: archive.zip
Files extracted to: /content/text_emotion_data


In [2]:
import pandas as pd

# Load the training data
train_file = "/content/text_emotion_data/data/train.tsv"
train_df = pd.read_csv(train_file, sep='\t')

# Show the first few rows and columns
print("Columns:", train_df.columns)
train_df.head()

Columns: Index(['My favourite food is anything I didn't have to cook myself.', '27',
       'eebbqej'],
      dtype='object')


,My favourite food is anything I didn't have to cook myself.,27,eebbqej
0,"Now if he does off himself, everyone will thin...",27,ed00q6i
1,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj
2,To make her feel threatened,14,ed7ypvh
3,Dirty Southern Wankers,3,ed0bdzj
4,OmG pEyToN iSn'T gOoD eNoUgH tO hElP uS iN tHe...,26,edvnz26


In [3]:
# Load emotions.txt to map index to emotion names
emotions_file = "/content/text_emotion_data/data/emotions.txt"
with open(emotions_file, 'r') as file:
    emotions = file.read().splitlines()

# Create dictionary mapping index to emotion
emotion_dict = {str(idx): label for idx, label in enumerate(emotions)}

print("Total emotions:", len(emotions))
print("Sample emotions:", emotions[:5])
print("Emotion mapping example:", emotion_dict['0'])


Total emotions: 28
Sample emotions: ['admiration', 'amusement', 'anger', 'annoyance', 'approval']
Emotion mapping example: admiration


In [4]:
# Map the emotion index to actual emotion names
train_df['emotion'] = train_df.iloc[:, 1].astype(str).map(emotion_dict)
train_df = train_df.rename(columns={train_df.columns[0]: 'text'})  # Rename the text column

# Show sample data
print("Sample data:")
print(train_df[['text', 'emotion']].head())

# Check distribution of emotions
print("\nEmotion distribution:")
print(train_df['emotion'].value_counts())

Sample data:
                                                text    emotion
0  Now if he does off himself, everyone will thin...    neutral
1                     WHY THE FUCK IS BAYLESS ISOING      anger
2                        To make her feel threatened       fear
3                             Dirty Southern Wankers  annoyance
4  OmG pEyToN iSn'T gOoD eNoUgH tO hElP uS iN tHe...   surprise

Emotion distribution:
emotion
neutral           12822
admiration         2710
approval           1873
gratitude          1857
amusement          1652
annoyance          1451
love               1427
disapproval        1402
curiosity          1389
anger              1025
optimism            861
confusion           858
joy                 853
sadness             817
surprise            720
disappointment      709
caring              649
realization         586
excitement          510
disgust             498
fear                430
desire              389
remorse             353
embarrassment       

In [5]:
import re

# Function to clean text
def clean_text(text):
    text = text.lower()  # Lowercase
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)  # Remove URLs
    text = re.sub(r"[^a-zA-Z\s]", '', text)  # Remove special characters
    text = re.sub(r"\s+", ' ', text).strip()  # Remove extra spaces
    return text

# Apply cleaning
train_df['clean_text'] = train_df['text'].apply(clean_text)

# Show sample cleaned data
train_df[['text', 'clean_text', 'emotion']].head()

,text,clean_text,emotion
0,"Now if he does off himself, everyone will thin...",now if he does off himself everyone will think...,neutral
1,WHY THE FUCK IS BAYLESS ISOING,why the fuck is bayless isoing,anger
2,To make her feel threatened,to make her feel threatened,fear
3,Dirty Southern Wankers,dirty southern wankers,annoyance
4,OmG pEyToN iSn'T gOoD eNoUgH tO hElP uS iN tHe...,omg peyton isnt good enough to help us in the ...,surprise


In [6]:
from transformers import DistilBertTokenizerFast

# Load the tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# Encode the text data
encoded_inputs = tokenizer(
    train_df['clean_text'].tolist(),
    truncation=True,
    padding=True,
    max_length=128,
    return_tensors="pt"
)

print("Sample input IDs:")
print(encoded_inputs['input_ids'][0])
print("Sample attention mask:")
print(encoded_inputs['attention_mask'][0])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Sample input IDs:
tensor([  101,  2085,  2065,  2002,  2515,  2125,  2370,  3071,  2097,  2228,
         2002,  2015,  2383,  1037,  4756, 29082,  2007,  2111,  2612,  1997,
         2941,  2757,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0])
Sample attention mask:
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0])


In [7]:
from sklearn.preprocessing import LabelEncoder
import torch

# Encode emotion labels into integers
label_encoder = LabelEncoder()
train_df['label'] = label_encoder.fit_transform(train_df['emotion'])

# Check the mapping from emotions to integers
label_mapping = dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))
print("Label mapping:")
print(label_mapping)

# Convert labels to torch tensor
labels = torch.tensor(train_df['label'].values)

# Check a sample
print("\nSample encoded labels:")
print(labels[:5])

Label mapping:
{'admiration': 0, 'amusement': 1, 'anger': 2, 'annoyance': 3, 'approval': 4, 'caring': 5, 'confusion': 6, 'curiosity': 7, 'desire': 8, 'disappointment': 9, 'disapproval': 10, 'disgust': 11, 'embarrassment': 12, 'excitement': 13, 'fear': 14, 'gratitude': 15, 'grief': 16, 'joy': 17, 'love': 18, 'nervousness': 19, 'neutral': 20, 'optimism': 21, 'pride': 22, 'realization': 23, 'relief': 24, 'remorse': 25, 'sadness': 26, 'surprise': 27, nan: 28}

Sample encoded labels:
tensor([20,  2, 14,  3, 27])


In [8]:
# Drop rows where the label is missing (nan)
train_df = train_df.dropna(subset=['emotion'])

# Re-encode after dropping missing values
train_df['label'] = label_encoder.fit_transform(train_df['emotion'])

# Update the label mapping
label_mapping = dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))
print("Updated label mapping:")
print(label_mapping)

# Convert labels to torch tensor
labels = torch.tensor(train_df['label'].values)

# Confirm dataset size
print("\nTotal samples after cleaning:", len(train_df))

Updated label mapping:
{'admiration': 0, 'amusement': 1, 'anger': 2, 'annoyance': 3, 'approval': 4, 'caring': 5, 'confusion': 6, 'curiosity': 7, 'desire': 8, 'disappointment': 9, 'disapproval': 10, 'disgust': 11, 'embarrassment': 12, 'excitement': 13, 'fear': 14, 'gratitude': 15, 'grief': 16, 'joy': 17, 'love': 18, 'nervousness': 19, 'neutral': 20, 'optimism': 21, 'pride': 22, 'realization': 23, 'relief': 24, 'remorse': 25, 'sadness': 26, 'surprise': 27}

Total samples after cleaning: 36307


In [9]:
from torch.utils.data import Dataset

class EmotionDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

# Create the dataset object
train_dataset = EmotionDataset(encoded_inputs, labels)

# Check a sample
print("Sample data from dataset:")
print(train_dataset[0])

Sample data from dataset:
{'input_ids': tensor([  101,  2085,  2065,  2002,  2515,  2125,  2370,  3071,  2097,  2228,
         2002,  2015,  2383,  1037,  4756, 29082,  2007,  2111,  2612,  1997,
         2941,  2757,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0]), 'labels': tensor(20)}


/tmp/ipython-input-2563364032.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


In [10]:
from transformers import DistilBertForSequenceClassification

# Define the model with the number of emotion classes
num_labels = len(label_mapping)  # 28 classes
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=num_labels)

print("Model loaded with", num_labels, "labels.")

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded with 28 labels.


In [11]:
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Split the data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_df['clean_text'].tolist(),
    train_df['label'].tolist(),
    test_size=0.2,  # 20% for validation
    random_state=42,
    stratify=train_df['label'].tolist() # Stratify to maintain emotion distribution
)

# Tokenize the split data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)

# Create Dataset objects
train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': train_labels
})

val_dataset = Dataset.from_dict({
    'input_ids': val_encodings['input_ids'],
    'attention_mask': val_encodings['attention_mask'],
    'labels': val_labels
})

# Combine into tokenized_datasets dictionary
tokenized_datasets = {
    'train': train_dataset,
    'validation': val_dataset
}

print("Training dataset size:", len(tokenized_datasets['train']))
print("Validation dataset size:", len(tokenized_datasets['validation']))

Training dataset size: 29045
Validation dataset size: 7262


In [12]:
!pip install --upgrade transformers accelerate

In [13]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.5 MB/s eta 0:00:00


In [14]:
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate
import os

# --- Define Evaluation Metric ---
# We still need this for the evaluation step AFTER training
accuracy_metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

# --- Bare-Bones Training Arguments ---
# We are removing all strategy-related arguments to avoid the bug.
training_args = TrainingArguments(
    output_dir="emotion_model_checkpoints",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_dir='./logs',
    # No evaluation_strategy, no save_strategy, no eval_steps, no load_best_model_at_end
)

# --- Simplified Trainer Initialization ---
# Note: We remove eval_dataset and compute_metrics for the training step.
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    # We will evaluate manually after training is done.
)

# --- Step 1: Train the Model ---
print("🚀 Starting model training with the simplest configuration...")
trainer.train()
print("🎉 Training complete!")

# --- Step 2: Save the Model ---
model_save_path = "emotion_model"
os.makedirs(model_save_path, exist_ok=True)
trainer.save_model(model_save_path)
print(f"\nModel saved to: {model_save_path}")


# --- Step 3: Manually Evaluate the Model ---
print("\n🧪 Now, evaluating the trained model on the validation set...")
eval_results = trainer.evaluate(
    eval_dataset=tokenized_datasets["validation"],
    metric_key_prefix="eval" # A standard prefix for evaluation results
)

print(f"\nValidation Results:\n{eval_results}")

🚀 Starting model training with the simplest configuration...


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sumangiri1964 (sumangiri1964-vellore-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,2.023000
1000,1.560100
1500,1.443200
2000,1.316000
2500,1.172400
3000,1.164200
3500,1.083400
4000,0.904400
4500,0.826400
5000,0.812900


🎉 Training complete!

Model saved to: emotion_model

🧪 Now, evaluating the trained model on the validation set...



Validation Results:
{'eval_loss': 1.419360876083374, 'eval_runtime': 9.2377, 'eval_samples_per_second': 786.127, 'eval_steps_per_second': 49.146, 'epoch': 3.0}


In [15]:
import evaluate

f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return f1_metric.compute(predictions=predictions, references=labels, average="weighted")

In [16]:
import evaluate
import numpy as np

# Load F1 metric
f1_metric = evaluate.load("f1")

# Define metric computation function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    # Weighted average accounts for class imbalance
    return f1_metric.compute(predictions=predictions, references=labels, average="weighted")

# Evaluate on validation set
eval_results = trainer.evaluate(
    eval_dataset=tokenized_datasets["validation"],
    metric_key_prefix="eval"
)

# Print results
print("\nValidation Results (F1-score included):")
print(eval_results)


Validation Results (F1-score included):
{'eval_loss': 1.419360876083374, 'eval_runtime': 10.5381, 'eval_samples_per_second': 689.122, 'eval_steps_per_second': 43.082, 'epoch': 3.0}


In [17]:
# After training
model.save_pretrained("emotion_model")
tokenizer.save_pretrained("emotion_model")  # THIS is important!

('emotion_model/tokenizer_config.json',
 'emotion_model/special_tokens_map.json',
 'emotion_model/vocab.txt',
 'emotion_model/added_tokens.json',
 'emotion_model/tokenizer.json')

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import numpy as np

model_path = "emotion_model"  # folder containing both model & tokenizer
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)  # now this will work

model.eval()
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [ ]:
import torch
import numpy as np
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import os

# --- Load Trained Model and Tokenizer ---
model_path = "emotion_model"  # path where you saved your model
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model.eval()
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# --- Emotion Mapping (indices -> labels) ---
# Use the same mapping you used in training
# For example: emotion_dict = {0: 'neutral', 1: 'admiration', ..., 27: 'grief'}
label_names = list(label_mapping.keys()) # Use the label_mapping generated earlier

# --- Function to Predict Emotion ---
def predict_emotion(texts):
    # Tokenize input
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Forward pass
    with torch.no_grad():
        outputs = model(**inputs)

    # Get predicted class indices
    preds = np.argmax(outputs.logits.detach().cpu().numpy(), axis=1)

    # Map indices to emotion labels
    predicted_emotions = [label_names[p] for p in preds]

    return predicted_emotions

# --- Test on Sample Texts ---
sample_texts = [
    "I am feeling really happy and excited today!",
    "I am so sad and lonely...",
    "I feel angry and frustrated with everything.",
    "I love her"
]

predictions = predict_emotion(sample_texts)
for text, emo in zip(sample_texts, predictions):
    print(f"Text: {text}\nPredicted Emotion: {emo}\n")

Text: I am feeling really happy and excited today!
Predicted Emotion: excitement

Text: I am so sad and lonely...
Predicted Emotion: sadness

Text: I feel angry and frustrated with everything.
Predicted Emotion: anger

Text: I love her
Predicted Emotion: love



In [ ]:
emotion_to_playlist = {
    "neutral": "https://open.spotify.com/playlist/5MX1quD2Hrs1I59eRTJ1Q8",         # Chill Hits
    "admiration": "https://open.spotify.com/playlist/37i9dQZF1DX2sUQwD7tbmL",       # Inspirational
    "approval": "https://open.spotify.com/playlist/37i9dQZF1DX4fpCWaHOned",        # Positive Vibes
    "gratitude": "https://open.spotify.com/playlist/37i9dQZF1DWU13kKnk03AP",       # Relaxed Acoustic
    "amusement": "https://open.spotify.com/playlist/37i9dQZF1DX8F067Zoxx2Z",        # Fun / Upbeat
    "annoyance": "https://open.spotify.com/playlist/06imAk4ddfj6jK7B7yRRX1?si=xF0Q0poFTKa5Avug4sw_2Q&pi=oDyTfyngTcauY",# Angry / Rock
    "love": "https://open.spotify.com/playlist/41YHbH6t9Pi7oEo0Gm0VUZ?si=f5a392f0aed64110",# Romantic
    "disapproval": "https://open.spotify.com/playlist/37i9dQZF1DX1KqNs3VtMZq",      # Intense / Angry
    "curiosity": "https://open.spotify.com/playlist/2VEEQQLJH3DcnmdnzFA5Bu?si=V25ypc2wT-O2vlFVAfkUZw", # Ambient / Chill
    "anger": "https://open.spotify.com/playlist/0KPEhXA3O9jHFtpd1Ix5OB",            # Rock / Angry
    "optimism": "https://open.spotify.com/playlist/37i9dQZF1DX3rxVfibe1L0",        # Mood Booster
    "confusion": "https://open.spotify.com/playlist/40j429vr0iYBkZeszSY7uB?si=PG43GtuHRgKVuiyMXNnYTA",       # Chill / Instrumental
    "joy": "https://open.spotify.com/playlist/4OmizGQKZhVEncdma1VdzO",             # Happy Hits
    "sadness": "https://open.spotify.com/playlist/7ABD15iASBIpPP5uJ5awvq",         # Sad / Melancholic
    "surprise": "https://open.spotify.com/playlist/3hstVxyKHTqOs2wlIl6RbI",        # Unexpected / Energetic
    "disappointment": "https://open.spotify.com/playlist/2XEg0YR4A91brqYPOdlkmC?si=hjfKzRMqSH6m_WCRCwW24g",  # Soft / Piano
    "caring": "https://open.spotify.com/playlist/2yAW8JNPAungPnkJ3J77As?si=vKGd9aVIRgua_yMTj3gCCQ",          # Gentle / Acoustic
    "realization": "https://open.spotify.com/playlist/0UG2VkubCyZbFhvVxCUmhD?si=Z5IH5bG0TLuGnsQKl_ngAg",     # Reflective / Chill
    "excitement": "https://open.spotify.com/playlist/37i9dQZF1DX3rxVfibe1L0",      # Party / Upbeat
    "disgust": "https://open.spotify.com/playlist/3qgzMg4m5tvf16PzlPgGa9",         # Intense / Dark
    "fear": "https://open.spotify.com/playlist/4uCuKjOagWPUgJgXQNcBUc",             # Calm Ambient
    "desire": "https://open.spotify.com/playlist/0FbMPhPu6ASm4xW5VoQYZd?si=6GhbmfZ7QhW-RscE6jKqlQ",          # Romantic / Passion
    "remorse": "https://open.spotify.com/playlist/6ilpOnJkqyr4yDkl44K6To?si=YG3ZUQIqRv2TELJ7yB1_Ig",         # Soft Piano / Emotional
    "embarrassment": "https://open.spotify.com/playlist/1JWg4tDnmoeJEO2LEaQ7aC?si=qqq17f9CRQazN7SVM3IgBA",   # Light / Fun
    "relief": "https://open.spotify.com/playlist/37i9dQZF1DWXe9gFZP0gtP?si=vrlnuOU4TOmwzOlpEnVzAw",          # Calm / Relaxing
    "nervousness": "https://open.spotify.com/track/4mwJh15tiWkoScwNwEHPHe?si=58969d48a2b24e50",     # Chill / Instrumental
    "pride": "https://open.spotify.com/playlist/37i9dQZF1DX2sUQwD7tbmL",           # Epic / Motivational
    "grief": "https://open.spotify.com/playlist/4VdAlUjMtVLMx8W24vCW9M?si=xZctcPdARmqKXKJz2UDgGw"             # Sad / Classical
}

In [ ]:
def suggest_playlist(texts):
    """
    Takes a list of texts, predicts their emotions, and returns suggested playlists.
    """
    # Predict emotions (assuming 'predicted_emotions' is a list of predicted emotions)
    predicted_emotions = ["joy", "anger", "sadness"]  # Example predictions

    # Map emotions to playlists
    playlists = [emotion_to_playlist.get(emotion, "https://open.spotify.com/playlist/37i9dQZF1DX3rxVfibe1L0") for emotion in predicted_emotions]

    return list(zip(texts, predicted_emotions, playlists))

# Example usage
sample_texts = [
    "I'm feeling really happy today!",
    "I'm so angry at the situation.",
    "I'm just neutral, not much going on."
]

results = suggest_playlist(sample_texts)
for text, emo, playlist in results:
    print(f"Text: {text}\nPredicted Emotion: {emo}\nSuggested Playlist: {playlist}\n")

Text: I'm feeling really happy today!
Predicted Emotion: joy
Suggested Playlist: https://open.spotify.com/playlist/4OmizGQKZhVEncdma1VdzO

Text: I'm so angry at the situation.
Predicted Emotion: anger
Suggested Playlist: https://open.spotify.com/playlist/0KPEhXA3O9jHFtpd1Ix5OB

Text: I'm just neutral, not much going on.
Predicted Emotion: sadness
Suggested Playlist: https://open.spotify.com/playlist/7ABD15iASBIpPP5uJ5awvq



In [ ]:
!pip install librosa torchaudio transformers soundfile

In [ ]:
from IPython.display import Audio
import soundfile as sf
import numpy as np

# Upload or record a file
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
    audio_path = fn

# Load audio
import librosa
y, sr = librosa.load(audio_path, sr=16000)  # resample to 16kHz
Audio(audio_path)

Saving man-laughing-03.wav to man-laughing-03.wav


In [ ]:
# Extract MFCC features
mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
mfccs_mean = np.mean(mfccs.T, axis=0)  # Average over time
print("MFCCs shape:", mfccs_mean.shape)

MFCCs shape: (40,)


In [ ]:
from transformers import AutoModelForAudioClassification
import torch
import librosa

# Device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load audio
audio_path = "/content/man-scream-01.wav"
y, sr = librosa.load(audio_path, sr=16000)
audio_tensor = torch.tensor(y).unsqueeze(0).to(device)  # batch dimension

# Load HuBERT-based SER model
model_name = "superb/hubert-large-superb-er"
model = AutoModelForAudioClassification.from_pretrained(model_name)
model.to(device)
model.eval()

# Predict
with torch.no_grad():
    logits = model(audio_tensor).logits

predicted_id = torch.argmax(logits, dim=-1).item()
predicted_emotion = model.config.id2label[predicted_id]

print("Predicted Emotion from Voice:", predicted_emotion)

Predicted Emotion from Voice: hap


In [ ]:
import librosa
import torch

def preprocess_audio(audio_path, target_sr=16000):
    # Load audio file
    audio, sr = librosa.load(audio_path, sr=target_sr)
    # Convert to tensor and add batch dimension
    audio_tensor = torch.tensor(audio).unsqueeze(0)
    return audio_tensor

audio_tensor = preprocess_audio("/content/man-laughing-03.wav")

In [ ]:
from transformers import AutoModelForAudioClassification

# Load the pre-trained model
model = AutoModelForAudioClassification.from_pretrained("superb/hubert-large-superb-er")
model.eval()

# Predict emotion
with torch.no_grad():
    logits = model(audio_tensor).logits

predicted_id = torch.argmax(logits, dim=-1).item()
predicted_emotion = model.config.id2label[predicted_id]

print("Predicted Emotion:", predicted_emotion)

Predicted Emotion: hap


In [ ]:
from google.colab import files
files.upload()  # Upload 'kaggle.json'

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"anubhaverma18","key":"90d1172b291f719deb2e908ac1157fdb"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d uwrfkaggler/ravdess-emotional-speech-audio
!unzip ravdess-emotional-speech-audio.zip -d ravdess_data

Dataset URL: https://www.kaggle.com/datasets/uwrfkaggler/ravdess-emotional-speech-audio
License(s): CC-BY-NC-SA-4.0
 94% 404M/429M [00:02<00:00, 189MB/s]
100% 429M/429M [00:02<00:00, 159MB/s]
Archive:  ravdess-emotional-speech-audio.zip
  inflating: ravdess_data/Actor_01/03-01-01-01-01-01-01.wav  
  inflating: ravdess_data/Actor_01/03-01-01-01-01-02-01.wav  
  inflating: ravdess_data/Actor_01/03-01-01-01-02-01-01.wav  
  inflating: ravdess_data/Actor_01/03-01-01-01-02-02-01.wav  
  inflating: ravdess_data/Actor_01/03-01-02-01-01-01-01.wav  
  inflating: ravdess_data/Actor_01/03-01-02-01-01-02-01.wav  
  inflating: ravdess_data/Actor_01/03-01-02-01-02-01-01.wav  
  inflating: ravdess_data/Actor_01/03-01-02-01-02-02-01.wav  
  inflating: ravdess_data/Actor_01/03-01-02-02-01-01-01.wav  
  inflating: ravdess_data/Actor_01/03-01-02-02-01-02-01.wav  
  inflating: ravdess_data/Actor_01/03-01-02-02-02-01-01.wav  
  inflating: ravdess_data/Actor_01/03-01-02-02-02-02-01.wav  
  inflating: ravdes

In [ ]:
import os
import librosa
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torchaudio.transforms as T

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# Emotion mapping based on RAVDESS
emotion_map = {
    '01': 'neutral',
    '02': 'calm',
    '03': 'happy',
    '04': 'sad',
    '05': 'angry',
    '06': 'fearful',
    '07': 'disgust',
    '08': 'surprised'
}

class RAVDESSDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        audio_path = self.file_paths[idx]
        label = self.labels[idx]
        waveform, sample_rate = librosa.load(audio_path, sr=16000)
        waveform = torch.tensor(waveform, dtype=torch.float32)
        if self.transform:
            waveform = self.transform(waveform)
        return waveform, label

In [ ]:
data_dir = 'ravdess_data/Actor_*'  # Adjust path as needed
file_paths = []
labels = []

# Gather all files and corresponding emotion labels
for root, _, files in os.walk('ravdess_data'):
    for file in files:
        if file.endswith(".wav"):
            path = os.path.join(root, file)
            emotion_code = file.split("-")[2]
            emotion = emotion_map.get(emotion_code, 'neutral')
            file_paths.append(path)
            labels.append(emotion)

# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Train-test split
train_paths, test_paths, train_labels, test_labels = train_test_split(
    file_paths, labels_encoded, test_size=0.2, random_state=42, stratify=labels_encoded
)

# Define transforms
transform = T.MelSpectrogram(sample_rate=16000, n_mels=40)

# Create datasets and loaders
train_dataset = RAVDESSDataset(train_paths, train_labels, transform=transform)
test_dataset = RAVDESSDataset(test_paths, test_labels, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

num_classes = len(label_encoder.classes_)
print(f"Number of classes: {num_classes}")


Number of classes: 8


In [ ]:
MAX_LEN = 400  # Adjust as needed

In [ ]:
class RAVDESSDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None, max_len=MAX_LEN):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform
        self.max_len = max_len

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        audio_path = self.file_paths[idx]
        label = self.labels[idx]

        # Load waveform
        waveform, sample_rate = librosa.load(audio_path, sr=16000)
        waveform = torch.tensor(waveform, dtype=torch.float32)

        # Apply transform
        if self.transform:
            spec = self.transform(waveform)
        else:
            spec = waveform.unsqueeze(0)

        # Pad or truncate along time dimension
        if spec.shape[1] > self.max_len:
            spec = spec[:, :self.max_len]
        else:
            padding = self.max_len - spec.shape[1]
            spec = torch.nn.functional.pad(spec, (0, padding))

        return spec, label

In [ ]:
class EmotionCNN(nn.Module):
    def __init__(self, num_classes):
        super(EmotionCNN, self).__init__()
        self.conv1 = nn.Conv1d(40, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool1d(2)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 200, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = EmotionCNN(num_classes=num_classes).to(device)

In [ ]:
train_dataset = RAVDESSDataset(train_paths, train_labels, transform=transform, max_len=MAX_LEN)
test_dataset = RAVDESSDataset(test_paths, test_labels, transform=transform, max_len=MAX_LEN)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
class EmotionCNN(nn.Module):
    def __init__(self, num_classes, max_len=MAX_LEN):
        super(EmotionCNN, self).__init__()
        self.conv1 = nn.Conv1d(40, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool1d(2)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=3, padding=1)

        # Calculate the size after pooling
        pooled_len = max_len // 2  # After first pooling
        pooled_len = pooled_len // 2  # After second pooling

        self.fc1 = nn.Linear(128 * pooled_len, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # Flatten
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
model = EmotionCNN(num_classes=num_classes, max_len=MAX_LEN).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")

Epoch 1/10, Loss: 2.0550, Accuracy: 28.30%
Epoch 2/10, Loss: 1.5502, Accuracy: 42.27%
Epoch 3/10, Loss: 1.3899, Accuracy: 50.56%
Epoch 4/10, Loss: 1.1840, Accuracy: 57.81%
Epoch 5/10, Loss: 0.9791, Accuracy: 64.63%
Epoch 6/10, Loss: 0.8320, Accuracy: 71.40%
Epoch 7/10, Loss: 1.0059, Accuracy: 71.01%
Epoch 8/10, Loss: 0.7849, Accuracy: 75.13%
Epoch 9/10, Loss: 0.6818, Accuracy: 78.52%
Epoch 10/10, Loss: 0.6091, Accuracy: 79.99%


In [ ]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 72.74%


In [ ]:
def predict_emotion(audio_path):
    waveform, sample_rate = librosa.load(audio_path, sr=16000)
    waveform = torch.tensor(waveform, dtype=torch.float32)

    # Apply transform (e.g., Mel Spectrogram)
    spec = transform(waveform)

    # Pad or truncate to MAX_LEN
    if spec.shape[1] > MAX_LEN:
        spec = spec[:, :MAX_LEN]
    else:
        padding = MAX_LEN - spec.shape[1]
        spec = torch.nn.functional.pad(spec, (0, padding))

    spec = spec.unsqueeze(0).to(device)  # Add batch dimension

    model.eval()
    with torch.no_grad():
        outputs = model(spec)
        _, predicted = torch.max(outputs, 1)

    return label_encoder.inverse_transform(predicted.cpu().numpy())[0]

In [ ]:
MAX_LEN = 400  # Must be same as in training

In [ ]:
model = EmotionCNN(num_classes=num_classes, max_len=MAX_LEN).to(device)

In [ ]:
# Example usage
predicted_emotion = predict_emotion('/content/baby-crying-64996.mp3')
print(f"Predicted Emotion: {predicted_emotion}")

Predicted Emotion: surprised
